## Test HF-Wrapper

In [1]:
import nest_asyncio
from backend.workflow.utils.hf_space_wrapper import CustomEmbedding

nest_asyncio.apply()

hf_space = CustomEmbedding()
resp = hf_space.embed_query("first input for testing")
print(resp)

[-0.026232652366161346, -0.037341125309467316, -0.013515371829271317, -0.09593390673398972, 0.021219264715909958, 0.01310790330171585, 0.009186848066747189, -0.0021368693560361862, -0.06100285053253174, 0.02155672013759613, -0.004443945363163948, -0.06014695763587952, 0.07706617563962936, -0.011318260803818703, -0.05833763629198074, 0.05761360749602318, -0.025580700486898422, 0.027943773195147514, 0.03623388707637787, -0.0069763073697686195, -0.004101807717233896, 0.0068744090385735035, -0.0244967769831419, 0.09954224526882172, -0.04477320611476898, -0.04187597334384918, -0.021219078451395035, 0.05838209018111229, 0.01302416529506445, -0.028194986283779144, 0.025503525510430336, 0.058818064630031586, -0.03324533998966217, -0.0482887327671051, -0.013044405728578568, -0.017613360658288002, -0.02800883911550045, 0.0010864975629374385, -0.027188662439584732, 0.008578338660299778, -0.03588852658867836, 0.02814428322017193, 0.05066215246915817, 0.012256486341357231, 0.011855210177600384, -0.

In [3]:
list_embed = hf_space.embed_documents(["query 1", "second long query aldsjflkajdlkfj", "third query: Samba Sada Shiv, MSV, Soorya, Amma, Appa"])
print(len(list_embed))
print(len(list_embed[0]))

3
1024


In [5]:
print(len(resp["response"]["data"]))
print(len(resp["response"]["data"][1]["embedding"]))

2
1024


## Data Ingestion

In [29]:
# file = "/home/msvelan/Programming/third-party/manim/docs/source/reference_index/utilities_misc.rst"
file = "/home/msvelan/Programming/third-party/manim/docs/source/guides/using_text.rst"

In [32]:
from langchain_community.document_loaders import UnstructuredRSTLoader

loader = UnstructuredRSTLoader(file_path=file, mode="elements")
docs = loader.load()

In [3]:
for i in range(len(docs)):
    print("----- Document ", i, " -----")
    print("Category: ", docs[i].metadata.get("category", "N/A"))
    print("Content: ", docs[i].page_content)

----- Document  0  -----
Category:  Title
Content:  Rendering Text and Formulas
----- Document  1  -----
Category:  NarrativeText
Content:  There are two different ways by which you can render Text in videos:
----- Document  2  -----
Category:  ListItem
Content:  Using Pango (~.text_mobject)
----- Document  3  -----
Category:  ListItem
Content:  Using LaTeX (~.tex_mobject)
----- Document  4  -----
Category:  NarrativeText
Content:  If you want to render simple text, you should use either ~.Text or ~.MarkupText, or one of its derivatives like ~.Paragraph. See using-text-objects for more information.
----- Document  5  -----
Category:  NarrativeText
Content:  LaTeX should be used when you need mathematical typesetting. See rendering-with-latex for more information.
----- Document  6  -----
Category:  Title
Content:  Text Without LaTeX
----- Document  7  -----
Category:  NarrativeText
Content:  The simplest way to add text to your animations is to use the ~.Text class. It uses the Pango l

In [10]:
print(docs[2].metadata)

{'source': '/home/msvelan/Programming/third-party/manim/docs/source/guides/using_text.rst', 'category_depth': 1, 'languages': ['eng'], 'file_directory': '/home/msvelan/Programming/third-party/manim/docs/source/guides', 'filename': 'using_text.rst', 'filetype': 'text/x-rst', 'last_modified': '2025-10-30T19:23:09', 'parent_id': 'e96af543cb0990f426fe30b79ebc4ef8', 'category': 'ListItem', 'element_id': '19d8eba8d58f93ddf569d0d2e23f5fdc'}


In [5]:
documentation_list = [doc.page_content for doc in docs if doc.metadata.get("category") != "UncategorizedText"]
documentation_str = "\n".join(documentation_list)
print(documentation_str)

Rendering Text and Formulas
There are two different ways by which you can render Text in videos:
Using Pango (~.text_mobject)
Using LaTeX (~.tex_mobject)
If you want to render simple text, you should use either ~.Text or ~.MarkupText, or one of its derivatives like ~.Paragraph. See using-text-objects for more information.
LaTeX should be used when you need mathematical typesetting. See rendering-with-latex for more information.
Text Without LaTeX
The simplest way to add text to your animations is to use the ~.Text class. It uses the Pango library to render text. With Pango, you can also render non-English alphabets like 你好 or こんにちは or 안녕하세요 or مرحبا بالعالم.
Here is a simple Hello World animation.
You can also use ~.MarkupText which allows the use of PangoMarkup (see the documentation of ~.MarkupText for details) to render text. For example:
Working with ~.Text
This section explains the properties of ~.Text and how can it be used in your animations.
Using Fonts
You can set a different 

### Group by titles (Deprecated: all subheadings also have category as Title)

In [ ]:
grouped_docs = []
current_section = None
for doc in docs:
    if doc.metadata["category"] == "UncategorizedText":
        continue
    if doc.metadata["category"] == "Title":
        if current_section:
            grouped_docs.append(current_section)
        current_section = {"title": doc.page_content, "content": []}
    else:
        current_section["content"].append(doc.page_content)
if current_section:
    grouped_docs.append(current_section)

# Combine text fields
final_docs = [
    "Title: " + d["title"] + "\n" + "\n".join(d["content"])
    for d in grouped_docs
]

for i, doc in enumerate(final_docs):
    print("="*30, f"Document {i+1}", "="*30)
    print(doc)


============================== Document 1 ==============================
Title: Rendering Text and Formulas
There are two different ways by which you can render Text in videos:
Using Pango (~.text_mobject)
Using LaTeX (~.tex_mobject)
If you want to render simple text, you should use either ~.Text or ~.MarkupText, or one of its derivatives like ~.Paragraph. See using-text-objects for more information.
LaTeX should be used when you need mathematical typesetting. See rendering-with-latex for more information.
============================== Document 2 ==============================
Title: Text Without LaTeX
The simplest way to add text to your animations is to use the ~.Text class. It uses the Pango library to render text. With Pango, you can also render non-English alphabets like 你好 or こんにちは or 안녕하세요 or مرحبا بالعالم.
Here is a simple Hello World animation.
You can also use ~.MarkupText which allows the use of PangoMarkup (see the documentation of ~.MarkupText for details) to render text.

### Group by title but with category depth

In [33]:
from langchain_core.documents import Document

grouped_docs = []
depth2title_map = {}
current_section = None
for doc in docs:
    if doc.metadata["category"] == "UncategorizedText":
        continue
    if doc.metadata["category"] == "Title":
        if current_section:
            grouped_docs.append(current_section)
        current_title = doc.page_content
        category_depth = doc.metadata["category_depth"]
        section_title = ""
        if category_depth >= 1:
            for depth in range(category_depth):
                section_title += depth2title_map.get(depth, "") + " > "
        current_section = {"title": section_title + current_title, "content": [],
                           "source": doc.metadata["source"],
                           "filename": doc.metadata["filename"]}
                           # source and filename are metadata
        depth2title_map[category_depth] = current_title
    else:
        current_section["content"].append(doc.page_content)
if current_section:
    grouped_docs.append(current_section)

def chunker(grouped_docs, max_tokens: int = 1000):
    """Returns list of Document object chunks for given grouped_docs"""

    def get_str(d: dict):
        """Returns page content for the grouped_docs object"""
        return "Title: " + d["title"] + "\n" + "\n".join(d["content"])
    
    def get_rolling_window_doc(rolling_window: str):
        """Returns Langchain Document object for rolling window chunk"""
        rolling_doc = Document(page_content=rolling_window, metadata={
            "source": grouped_docs[i]["source"],
            "filename": grouped_docs[i]["filename"],
            "type": "documentation"
        })
        return rolling_doc
        

    new_groups = []
    rolling_window = get_str(grouped_docs[0])
    cur_size = len(rolling_window)
    base_title = grouped_docs[0]["title"]

    for i in range(1, len(grouped_docs)):
        cur_st = get_str(grouped_docs[i])
        cur_title = grouped_docs[i]["title"]
        if cur_title.startswith(base_title):
            if cur_size+len(cur_st) > max_tokens:
                rolling_window_doc = get_rolling_window_doc(rolling_window)
                rolling_window_doc.metadata.update({"title": base_title})
                new_groups.append(rolling_window_doc)
                rolling_window = cur_st
            else:
                rolling_window += "\n\n"
                rolling_window += cur_st
        else:
            rolling_window_doc = get_rolling_window_doc(rolling_window)
            rolling_window_doc.metadata.update({"title": base_title})
            new_groups.append(rolling_window_doc)
            rolling_window = cur_st
            base_title = cur_title

    if rolling_window:
        rolling_window_doc = get_rolling_window_doc(rolling_window)
        rolling_window_doc.metadata.update({"title": base_title})
        new_groups.append(rolling_window_doc)

    return new_groups

final_docs = chunker(grouped_docs)
print(len(final_docs))
for i, doc in enumerate(final_docs):
    print("="*30, f"Document {i+1}", "="*30)
    print(doc.page_content)
    print()
    print("Metadata:")
    for k, v in doc.metadata.items():
        print(k, ":", v)

4
============================== Document 1 ==============================
Title: Rendering Text and Formulas
There are two different ways by which you can render Text in videos:
Using Pango (~.text_mobject)
Using LaTeX (~.tex_mobject)
If you want to render simple text, you should use either ~.Text or ~.MarkupText, or one of its derivatives like ~.Paragraph. See using-text-objects for more information.
LaTeX should be used when you need mathematical typesetting. See rendering-with-latex for more information.

Title: Rendering Text and Formulas > Text Without LaTeX
The simplest way to add text to your animations is to use the ~.Text class. It uses the Pango library to render text. With Pango, you can also render non-English alphabets like 你好 or こんにちは or 안녕하세요 or مرحبا بالعالم.
Here is a simple Hello World animation.
You can also use ~.MarkupText which allows the use of PangoMarkup (see the documentation of ~.MarkupText for details) to render text. For example:

Title: Rendering Text and

### Embed using CustomEmbedding

In [8]:
from backend.workflow.utils.hf_space_wrapper import CustomEmbedding

hf_space = CustomEmbedding()
doc_embedding = hf_space.embed_documents([doc.page_content for doc in final_docs])
print(len(doc_embedding))
print(len(doc_embedding[0]))

4
1024


## Using Regex to parse the code blocks

In [60]:
import os
import re
from textwrap import dedent
from typing import List, Tuple


def extract_manim_blocks(rst_text: str, source: str, max_tokens=1000) -> List:
    """
    Parse an .rst / Markdown string and return a list of tuples:

        (class_name, python_source, options)

    * ``class_name`` - the class name from the directive line (e.g., 'ExampleTransform')
    * ``python_source`` - the code belonging to the ``.. manim::`` block,
      de-indented so it can be executed or written to a .py file.
    * ``options`` - a list of the raw option lines (e.g. ':ref_classes: Transform').

    The function works for any number of blocks in the same file.
    """

    filename = os.path.basename(source)
    
    def chunker(blocks, max_tokens: int = 1000):
        """Returns list of Document object chunks for given grouped_docs"""

        def get_str(block: tuple):
            """Returns page content for the manim block object"""
            _, code, opts = block
            content = ""
            if opts:
                content = "References:\n"
                for o in opts:
                    if o:
                        content += f"  {o}\n"
            content += "Code:\n"
            content += f"{code}"
            return content
        
        def get_rolling_window_doc(rolling_window: str, references):
            """Returns Langchain Document object for rolling window chunk"""
            rolling_doc = Document(page_content=rolling_window, metadata={
                "source": source,
                "filename": filename,
                "type": "code"
            })
            ref_classes = " ".join(references["ref_classes"])
            ref_functions = " ".join(references["ref_functions"])
            ref_methods = " ".join(references["ref_methods"])
            rolling_doc.metadata.update({
                "ref_classes": ref_classes,
                "ref_functions": ref_functions,
                "ref_methods": ref_methods
            })
            return rolling_doc
        
        def update_references(cur_references, block):
            _, _, opts = block
            if opts:
                for o in opts:
                    _, pred, ref_names = o.split(":") 
                    ref_names = ref_names.strip()
                    ref_names = set(ref_names.split())  # set of unique references
                    if pred == "ref_classes":
                        ns = cur_references["ref_classes"].union(ref_names)
                        cur_references["ref_classes"] = ns
                    elif pred == "ref_functions":
                        ns = cur_references["ref_functions"].union(ref_names)
                        cur_references["ref_functions"] = ns
                    elif pred == "ref_methods":
                        ns = cur_references["ref_methods"].union(ref_names)
                        cur_references["ref_methods"] = ns
            

        new_groups = []
        rolling_window = get_str(blocks[0])
        cur_size = len(rolling_window)
        cur_references = {
            "ref_classes": set(),
            "ref_functions": set(),
            "ref_methods": set()
        }
        update_references(cur_references, blocks[0])

        for i in range(1, len(blocks)):
            cur_st = get_str(blocks[i])
            if cur_size+len(cur_st) > max_tokens:
                rolling_window_doc = get_rolling_window_doc(rolling_window,
                                                            cur_references)
                new_groups.append(rolling_window_doc)
                rolling_window = cur_st
                cur_references.update(dict.fromkeys(cur_references, set()))
            else:
                rolling_window += "\n\n"
                rolling_window += cur_st
                update_references(cur_references, blocks[i])

        if rolling_window:
            rolling_window_doc = get_rolling_window_doc(rolling_window, cur_references)
            new_groups.append(rolling_window_doc)

        return new_groups

    
    pattern = re.compile(
        r"""(?mx)                                      # multiline + verbose
        ^\.\.\s+manim::\s+(?P<cls_name>\S+)\s*\n      # directive line with class name
        (?P<options>(?:[ ]{4,}:[^\n]*\n)*)             # option lines (4+ spaces, start with :)
        (?:[ ]*\n)                                     # blank line between options and code
        (?P<code>(?:[ ]{4,}[^\n]*\n|[ ]*\n)+)   # lines with 4+ spaces OR empty lines
        """
    )

    blocks: List[Tuple[str, str, List[str]]] = []

    for m in pattern.finditer(rst_text):
        # ----- class name ----------------------------------------------
        cls_name = m.group('cls_name')
        
        # ----- options -------------------------------------------------
        raw_opts = m.group('options')
        # Extract and clean option lines
        opts = [line.strip() for line in raw_opts.splitlines() if line.strip().startswith(':ref')]

        # ----- code ----------------------------------------------------
        raw_code = m.group('code')
        raw_code = raw_code.strip("\n")
        # Remove the common leading indentation
        code = dedent(raw_code)

        blocks.append((cls_name, code, opts))
    
    return chunker(blocks, max_tokens)

# Test with the sample data
sample_rst = r"""
.. manim:: ExampleTransform
    :ref_classes: Transform
    :ref_methods: Scene.play

    class ExampleTransform(Scene):
        def construct(self):
            self.camera.background_color = WHITE
            m1 = Square().set_color(RED)
            m2 = Rectangle().set_color(RED).rotate(0.2)
            self.play(Transform(m1,m2))

The Transform function maps points of the previous mobject to the points of the next mobject.

.. manim:: ExampleRotation
    :ref_classes: Rotation
    :ref_methods: Scene.play

    class ExampleRotation(Scene):
        def construct(self):
            self.camera.background_color = WHITE
            m1a = Square().set_color(RED).shift(LEFT)
            m1b = Circle().set_color(RED).shift(LEFT)
            m2a = Square().set_color(BLUE).shift(RIGHT)
            m2b = Circle().set_color(BLUE).shift(RIGHT)
            points = m2a.points
            points = np.roll(points, int(len(points)/4), axis=0)
            m2a.points = points
            self.play(Transform(m1a,m1b),Transform(m2a,m2b), run_time=1)

## Scenes

.. manim:: CountingScene
    :ref_classes: Animation DecimalNumber
    :ref_methods: Animation.interpolate_mobject Scene.play

    class Count(Animation):
        def __init__(self, number: DecimalNumber, start: float, end: float, **kwargs) -> None:
            # Pass number as the mobject of the animation
            super().__init__(number,  **kwargs)
            # Set start and end
            self.start = start
            self.end = end
"""

blocks = extract_manim_blocks(sample_rst, "testing-sample")

def pretty_print_blocks(blocks):
    for i, doc in enumerate(blocks, 1):
        print("="*30, f"Block #{i}", "="*30)
        print(doc.page_content)
        print("Metadata:")
        for k,v in doc.metadata.items():
            print(k, ": ", v)

# pretty_print_blocks(blocks)
print(blocks)
print(len(blocks))

[Document(metadata={'source': 'testing-sample', 'filename': 'testing-sample', 'type': 'code', 'ref_classes': 'Rotation Transform DecimalNumber Animation', 'ref_functions': '', 'ref_methods': 'Animation.interpolate_mobject Scene.play'}, page_content='References:\n  :ref_classes: Transform\n  :ref_methods: Scene.play\nCode:\nclass ExampleTransform(Scene):\n    def construct(self):\n        self.camera.background_color = WHITE\n        m1 = Square().set_color(RED)\n        m2 = Rectangle().set_color(RED).rotate(0.2)\n        self.play(Transform(m1,m2))\n\nReferences:\n  :ref_classes: Rotation\n  :ref_methods: Scene.play\nCode:\nclass ExampleRotation(Scene):\n    def construct(self):\n        self.camera.background_color = WHITE\n        m1a = Square().set_color(RED).shift(LEFT)\n        m1b = Circle().set_color(RED).shift(LEFT)\n        m2a = Square().set_color(BLUE).shift(RIGHT)\n        m2b = Circle().set_color(BLUE).shift(RIGHT)\n        points = m2a.points\n        points = np.roll(po

In [61]:
file = "/home/msvelan/Programming/third-party/manim/docs/source/guides/using_text.rst"
with open(file, "r") as f:
    rst_content = f.read()

blocks = extract_manim_blocks(rst_content, file, max_tokens=500)
print(len(blocks))
# print(blocks)
pretty_print_blocks(blocks)

3
============================== Block #1 ==============================
References:
  :ref_classes: Text
Code:
class HelloWorld(Scene):
    def construct(self):
        text = Text("Hello world", font_size=144)
        self.add(text)

References:
  :ref_classes: MarkupText
Code:
class SingleLineColor(Scene):
    def construct(self):
        text = MarkupText(
            f'all in red <span fgcolor="{YELLOW}">except this</span>', color=RED
        )
        self.add(text)

Code:
class FontsExample(Scene):
    def construct(self):
        ft = Text("Noto Sans", font="Noto Sans")
        self.add(ft)

Code:
class SlantsExample(Scene):
    def construct(self):
        a = Text("Italic", slant=ITALIC)
        self.add(a)
Metadata:
source :  /home/msvelan/Programming/third-party/manim/docs/source/guides/using_text.rst
filename :  using_text.rst
type :  code
ref_classes :  MarkupText Text
ref_functions :  
ref_methods :  
============================== Block #2 ==============================

### Creating Document from python files

In [17]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(metadata={}, page_content='def hello_world():\n    print("Hello, World!")'),
 Document(metadata={}, page_content='# Call the function\nhello_world()')]

In [18]:
import os

from dotenv import load_dotenv

load_dotenv()

MANIM_DIR = os.getenv("MANIM_DIR", "")

basic_scene_py = ""
base_path_basic_scene = "example_scenes/basic.py"
abs_path_basic_scene = os.path.join(MANIM_DIR, base_path_basic_scene)
with open(abs_path_basic_scene) as f:
    basic_scene_py = f.read()

print(basic_scene_py)

#!/usr/bin/env python


from manim import *

# To watch one of these scenes, run the following:
# python --quality m manim -p example_scenes.py SquareToCircle
#
# Use the flag --quality l for a faster rendering at a lower quality.
# Use -s to skip to the end and just save the final frame
# Use the -p to have preview of the animation (or image, if -s was
# used) pop up once done.
# Use -n <number> to skip ahead to the nth animation of a scene.
# Use -r <number> to specify a resolution (for example, -r 1920,1080
# for a 1920x1080 video)


class OpeningManim(Scene):
    def construct(self):
        title = Tex(r"This is some \LaTeX")
        basel = MathTex(r"\sum_{n=1}^\infty \frac{1}{n^2} = \frac{\pi^2}{6}")
        VGroup(title, basel).arrange(DOWN)
        self.play(
            Write(title),
            FadeIn(basel, shift=DOWN),
        )
        self.wait()

        transform_title = Tex("That was a transform")
        transform_title.to_corner(UP + LEFT)
        self.play(
       

In [19]:
advanced_scene_py = ""
base_path_advanced_scene = "example_scenes/advanced_tex_fonts.py"
abs_path_advanced_scene = os.path.join(MANIM_DIR, base_path_advanced_scene)
with open(abs_path_advanced_scene) as f:
    advanced_scene_py = f.read()

print(advanced_scene_py)

from manim import *

# French Cursive LaTeX font example from http://jf.burnol.free.fr/showcase.html

# Example 1 Manually creating a Template

TemplateForFrenchCursive = TexTemplate(
    preamble=r"""
\usepackage[english]{babel}
\usepackage{amsmath}
\usepackage{amssymb}
\usepackage[T1]{fontenc}
\usepackage[default]{frcursive}
\usepackage[eulergreek,noplusnominus,noequal,nohbar,%
nolessnomore,noasterisk]{mathastext}
""",
)


def FrenchCursive(*tex_strings, **kwargs):
    return Tex(*tex_strings, tex_template=TemplateForFrenchCursive, **kwargs)


class TexFontTemplateManual(Scene):
    """An example scene that uses a manually defined TexTemplate() object to create
    LaTeX output in French Cursive font
    """

    def construct(self):
        self.add(Tex("Tex Font Example").to_edge(UL))
        self.play(Create(FrenchCursive("$f: A \\longrightarrow B$").shift(UP)))
        self.play(Create(FrenchCursive("Behold! We can write math in French Cursive")))
        self.wait(1)
        sel

In [20]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000
)
python_docs = python_splitter.create_documents([basic_scene_py, advanced_scene_py])
python_docs


[Document(metadata={}, page_content='#!/usr/bin/env python\n\n\nfrom manim import *\n\n# To watch one of these scenes, run the following:\n# python --quality m manim -p example_scenes.py SquareToCircle\n#\n# Use the flag --quality l for a faster rendering at a lower quality.\n# Use -s to skip to the end and just save the final frame\n# Use the -p to have preview of the animation (or image, if -s was\n# used) pop up once done.\n# Use -n <number> to skip ahead to the nth animation of a scene.\n# Use -r <number> to specify a resolution (for example, -r 1920,1080\n# for a 1920x1080 video)'),
 Document(metadata={}, page_content='class OpeningManim(Scene):\n    def construct(self):\n        title = Tex(r"This is some \\LaTeX")\n        basel = MathTex(r"\\sum_{n=1}^\\infty \\frac{1}{n^2} = \\frac{\\pi^2}{6}")\n        VGroup(title, basel).arrange(DOWN)\n        self.play(\n            Write(title),\n            FadeIn(basel, shift=DOWN),\n        )\n        self.wait()\n\n        transform_t

In [21]:
def pretty_print_pycode(docs):
    for i, doc in enumerate(docs, 1):
        print("="*30, f"Block {i}", "="*30)
        print(doc.page_content)
        print("Metadata")
        print(doc.metadata)

pretty_print_pycode(python_docs)

============================== Block 1 ==============================
#!/usr/bin/env python


from manim import *

# To watch one of these scenes, run the following:
# python --quality m manim -p example_scenes.py SquareToCircle
#
# Use the flag --quality l for a faster rendering at a lower quality.
# Use -s to skip to the end and just save the final frame
# Use the -p to have preview of the animation (or image, if -s was
# used) pop up once done.
# Use -n <number> to skip ahead to the nth animation of a scene.
# Use -r <number> to specify a resolution (for example, -r 1920,1080
# for a 1920x1080 video)
Metadata
{}
============================== Block 2 ==============================
class OpeningManim(Scene):
    def construct(self):
        title = Tex(r"This is some \LaTeX")
        basel = MathTex(r"\sum_{n=1}^\infty \frac{1}{n^2} = \frac{\pi^2}{6}")
        VGroup(title, basel).arrange(DOWN)
        self.play(
            Write(title),
            FadeIn(basel, shift=DOWN),
      

### Storing in a VectorStore

In [37]:
import nest_asyncio
from backend.workflow.utils.hf_space_wrapper import CustomEmbedding
from langchain_core.vectorstores import InMemoryVectorStore

nest_asyncio.apply()

hf_space = CustomEmbedding()

vector_store = InMemoryVectorStore(hf_space)
ids = vector_store.add_documents(python_docs)
print(ids)

['ba740242-ebd4-4d85-acb9-423cfdc217b2', '5629c139-669f-4a2c-9b9a-73501cc447b0', '7a8d3a2a-792e-467d-ad6e-99ca96156267', 'b11065f3-0978-45b1-8179-262105b5e185', 'fd161b5d-719d-4e5d-8798-fc37023b5151', '524ba6ba-2ebd-4a9d-a480-b707fa577042', '2a949c0b-9ad9-41ed-a595-50a1b4a2ff8c', '7cbc971a-39d3-4d2e-9339-3dfec27c70cf', '659f796d-ec4e-4da4-a181-3d577006b7f9', '43147ae0-5543-401c-9f2d-1ed7cac0ad32']


In [38]:
print(vector_store.add_documents(final_docs))

['629c950f-6b5f-4cae-a0d8-8980211ec03e', 'c0b42c89-0556-4126-b214-6fea4d75daac', '28b1328e-310f-4324-9681-17da36814e9a', '525ccc68-8d28-4dbe-b991-5a11b171da14']


In [ ]:
def get_detailed_instruct(query: str) -> str:
    task_description="""
Given a query, retrieve relevant code snippets that answer the query
"""
    return f'Instruct: {task_description}\nQuery:{query}'

results = vector_store.similarity_search_with_score(
    get_detailed_instruct("How to write text in screen in a beautiful way? Prefer fetching code snippets and examples.")
)

print(results)

[(Document(id='629c950f-6b5f-4cae-a0d8-8980211ec03e', metadata={'source': '/home/msvelan/Programming/third-party/manim/docs/source/guides/using_text.rst', 'filename': 'using_text.rst', 'type': 'documentation', 'title': 'Rendering Text and Formulas'}, page_content='Title: Rendering Text and Formulas\nThere are two different ways by which you can render Text in videos:\nUsing Pango (~.text_mobject)\nUsing LaTeX (~.tex_mobject)\nIf you want to render simple text, you should use either ~.Text or ~.MarkupText, or one of its derivatives like ~.Paragraph. See using-text-objects for more information.\nLaTeX should be used when you need mathematical typesetting. See rendering-with-latex for more information.\n\nTitle: Rendering Text and Formulas > Text Without LaTeX\nThe simplest way to add text to your animations is to use the ~.Text class. It uses the Pango library to render text. With Pango, you can also render non-English alphabets like 你好 or こんにちは or 안녕하세요 or مرحبا بالعالم.\nHere is a simp

In [50]:
print(len(results))
for result in results:
    print(f"Score: {result[1]}")
pretty_print_pycode([doc for doc, _ in results])

4
Score: 0.5013535671508345
Score: 0.4837276339151001
Score: 0.4487631884870763
Score: 0.44707076696433945
============================== Block 1 ==============================
Title: Rendering Text and Formulas
There are two different ways by which you can render Text in videos:
Using Pango (~.text_mobject)
Using LaTeX (~.tex_mobject)
If you want to render simple text, you should use either ~.Text or ~.MarkupText, or one of its derivatives like ~.Paragraph. See using-text-objects for more information.
LaTeX should be used when you need mathematical typesetting. See rendering-with-latex for more information.

Title: Rendering Text and Formulas > Text Without LaTeX
The simplest way to add text to your animations is to use the ~.Text class. It uses the Pango library to render text. With Pango, you can also render non-English alphabets like 你好 or こんにちは or 안녕하세요 or مرحبا بالعالم.
Here is a simple Hello World animation.
You can also use ~.MarkupText which allows the use of PangoMarkup (see 

In [ ]:
# NOTE: Generate 10 most similar documents for a given query,
# take 3 documents of type: code and 2 documents of type: documentation.

# NOTE: Other method can be to have an LLM model decide which documents to choose
# from the top 10 most similar documents given their metadata information as well.

### Connecting to Neon's Postgres with PGVector enabled

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
TABLE_NAME = os.getenv("TABLE_NAME")
VECTOR_SIZE = os.getenv("VECTOR_SIZE")

CONNECTION_STRING = (
    f"postgresql+asyncpg://{POSTGRES_USER}:{POSTGRES_HOST}"
    f":{POSTGRES_PORT}/{POSTGRES_DB}"
)

from langchain_postgres import PGEngine

pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)

In [2]:
await pg_engine.ainit_vectorstore_table(
    table_name=TABLE_NAME,
    vector_size=VECTOR_SIZE,
)

ProgrammingError: (sqlalchemy.dialects.postgresql.asyncpg.ProgrammingError) <class 'asyncpg.exceptions.DuplicateTableError'>: relation "manim_docs" already exists
[SQL: CREATE TABLE "public"."manim_docs"(
            "langchain_id" UUID PRIMARY KEY,
            "content" TEXT NOT NULL,
            "embedding" vector(1024) NOT NULL
            ,
"langchain_metadata" JSON
);]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
from backend.workflow.utils.hf_space_wrapper import CustomEmbedding

hf_space = CustomEmbedding()  # embedding service

from langchain_postgres import PGVectorStore

store = await PGVectorStore.create(
    engine=pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=hf_space,
)

In [22]:
# now we can use this store to add documents
await store.aadd_documents(python_docs)

['11bd9e46-5d3e-430a-8811-a17678072c54',
 'f3c67502-99c0-4548-b7fa-c7b804824728',
 'fcc0209e-0dbb-4972-8da6-7c75b571918c',
 '4aebdea4-7f24-4307-a25d-a82b0fc535da',
 '20a019c5-e8f1-4860-b2ec-fa2fcd4e578c',
 '1154253b-c8f1-4b65-9df3-41d98cd8a1ac',
 '7bfe5835-c947-4298-92c5-0bc153331b87',
 '22aca00c-5d07-4b20-b8c8-a50036978af2',
 '9023823d-e20b-4b66-88ac-f130fff83073',
 'f18d5e88-f0d9-46bd-a29f-ee6311b18ef6']

In [36]:
await store.aadd_documents(final_docs)

['fb2c1b5b-1cb5-41f4-b957-2b2fe6b3383d',
 'b71e815f-896d-45e2-8b66-caa9caece197',
 'ab4ef035-adc9-41db-9dc4-dbee603e5804',
 '624373fa-dc9f-48b7-ac68-e2873264404c']

In [37]:
# query vector store
# Query Instruction Template for Qwen3-Embedding:0.6B model
def get_detailed_instruct(query: str) -> str:
    task_description="""
Given a query, retrieve relevant code snippets that answer the query
"""
    return f'Instruct: {task_description}\nQuery:{query}'

query = get_detailed_instruct("Create a basic scene where you add hello world text\
 in the screen")

docs = await store.asimilarity_search(query)
print(docs)

[Document(id='11bd9e46-5d3e-430a-8811-a17678072c54', metadata={}, page_content='#!/usr/bin/env python\n\n\nfrom manim import *\n\n# To watch one of these scenes, run the following:\n# python --quality m manim -p example_scenes.py SquareToCircle\n#\n# Use the flag --quality l for a faster rendering at a lower quality.\n# Use -s to skip to the end and just save the final frame\n# Use the -p to have preview of the animation (or image, if -s was\n# used) pop up once done.\n# Use -n <number> to skip ahead to the nth animation of a scene.\n# Use -r <number> to specify a resolution (for example, -r 1920,1080\n# for a 1920x1080 video)'), Document(id='fcc0209e-0dbb-4972-8da6-7c75b571918c', metadata={}, page_content='class WarpSquare(Scene):\n    def construct(self):\n        square = Square()\n        self.play(\n            ApplyPointwiseFunction(\n                lambda point: complex_to_R3(np.exp(R3_to_complex(point))),\n                square,\n            ),\n        )\n        self.wait()

In [38]:
for i, doc in enumerate(docs, 1):
    print("="*30, f"Block {i}", "="*30)
    print(doc.page_content)

============================== Block 1 ==============================
#!/usr/bin/env python


from manim import *

# To watch one of these scenes, run the following:
# python --quality m manim -p example_scenes.py SquareToCircle
#
# Use the flag --quality l for a faster rendering at a lower quality.
# Use -s to skip to the end and just save the final frame
# Use the -p to have preview of the animation (or image, if -s was
# used) pop up once done.
# Use -n <number> to skip ahead to the nth animation of a scene.
# Use -r <number> to specify a resolution (for example, -r 1920,1080
# for a 1920x1080 video)
============================== Block 2 ==============================
class WarpSquare(Scene):
    def construct(self):
        square = Square()
        self.play(
            ApplyPointwiseFunction(
                lambda point: complex_to_R3(np.exp(R3_to_complex(point))),
                square,
            ),
        )
        self.wait()


class WriteStuff(Scene):
    def construct

### Get Summary elements from rst files

In [4]:
from backend.workflow.utils.ingest_docs import _get_all_documents

file = "/home/msvelan/Programming/third-party/manim/docs/source/reference.rst"
docs = _get_all_documents(file)

In [5]:
for i, doc in enumerate(docs, 1):
    print("="*30, f"Block {i}", "="*30)
    print(doc.page_content)

============================== Block 1 ==============================
Title: Reference Manual
This reference manual details modules, functions, and variables included in Manim, describing what they are and what they do. For learning how to use Manim, see tutorials/index. For a list of changes since the last release, see the changelog.
The pages linked to here are currently a work in progress.

Title: Reference Manual > Inheritance Graphs


Title: Reference Manual > Inheritance Graphs > Animations


Title: Reference Manual > Inheritance Graphs > Cameras


Title: Reference Manual > Inheritance Graphs > Mobjects


Title: Reference Manual > Inheritance Graphs > Scenes


Title: Reference Manual > Module Index



In [18]:
import re
from textwrap import dedent


def _get_other_directive_blocks(rst_text: str):
    """Extract all non-manim directives"""
    pattern = re.compile(
        r"""(?mx)
        ^\.\.\s+(?!manim::)(?!toctree::)(?!currentmodule::)(?P<directive>\S+)\s*\n
        (?P<content>(?:[ ]{3,}[^\n]*\n|[ ]*\n)*)
        """
    )
    blocks: List[Tuple[str, str]] = []
    for m in pattern.finditer(rst_text):
        directive = m.group("directive")
        raw_content = m.group("content")
        content = dedent(raw_content).rstrip("\n")
        blocks.append((directive, content))
    return blocks

file = "/home/msvelan/Programming/third-party/manim/docs/source/reference.rst"
with open(file) as f:
    rst_text = f.read()

blocks = _get_other_directive_blocks(rst_text)

In [19]:
blocks

[('inheritance-diagram::',
  'manim.animation.animation\nmanim.animation.changing\nmanim.animation.composition\nmanim.animation.creation\nmanim.animation.fading\nmanim.animation.growing\nmanim.animation.indication\nmanim.animation.movement\nmanim.animation.numbers\nmanim.animation.rotation\nmanim.animation.specialized\nmanim.animation.speedmodifier\nmanim.animation.transform\nmanim.animation.transform_matching_parts\nmanim.animation.updaters.mobject_update_utils\nmanim.animation.updaters.update\n:parts: 1\n:top-classes: manim.animation.animation.Animation'),
 ('inheritance-diagram::',
  'manim.camera.camera\nmanim.camera.mapping_camera\nmanim.camera.moving_camera\nmanim.camera.multi_camera\nmanim.camera.three_d_camera\n:parts: 1\n:top-classes: manim.camera.camera.Camera, manim.mobject.mobject.Mobject'),
 ('inheritance-diagram::',
  'manim.mobject.frame\nmanim.mobject.geometry.arc\nmanim.mobject.geometry.boolean_ops\nmanim.mobject.geometry.line\nmanim.mobject.geometry.polygram\nmanim.mo

In [20]:
for block in blocks:
    print(block[1])
    print("="*30)

manim.animation.animation
manim.animation.changing
manim.animation.composition
manim.animation.creation
manim.animation.fading
manim.animation.growing
manim.animation.indication
manim.animation.movement
manim.animation.numbers
manim.animation.rotation
manim.animation.specialized
manim.animation.speedmodifier
manim.animation.transform
manim.animation.transform_matching_parts
manim.animation.updaters.mobject_update_utils
manim.animation.updaters.update
:parts: 1
:top-classes: manim.animation.animation.Animation
manim.camera.camera
manim.camera.mapping_camera
manim.camera.moving_camera
manim.camera.multi_camera
manim.camera.three_d_camera
:parts: 1
:top-classes: manim.camera.camera.Camera, manim.mobject.mobject.Mobject
manim.mobject.frame
manim.mobject.geometry.arc
manim.mobject.geometry.boolean_ops
manim.mobject.geometry.line
manim.mobject.geometry.polygram
manim.mobject.geometry.shape_matchers
manim.mobject.geometry.tips
manim.mobject.graph
manim.mobject.graphing.coordinate_systems
mani

In [26]:
from langchain_core.documents import Document


# Now I can just create Document objects with each block.
def create_summary_docs(blocks):
    docs = []
    for directive, content in blocks:
        t = directive.rstrip(":")
        doc = Document(content, metadata={
            "type": "summary",
            "supplement": t,
        })
        docs.append(doc)
    return docs

docs = create_summary_docs(blocks)

In [27]:
docs

[Document(metadata={'type': 'summary', 'supplement': 'inheritance-diagram'}, page_content='manim.animation.animation\nmanim.animation.changing\nmanim.animation.composition\nmanim.animation.creation\nmanim.animation.fading\nmanim.animation.growing\nmanim.animation.indication\nmanim.animation.movement\nmanim.animation.numbers\nmanim.animation.rotation\nmanim.animation.specialized\nmanim.animation.speedmodifier\nmanim.animation.transform\nmanim.animation.transform_matching_parts\nmanim.animation.updaters.mobject_update_utils\nmanim.animation.updaters.update\n:parts: 1\n:top-classes: manim.animation.animation.Animation'),
 Document(metadata={'type': 'summary', 'supplement': 'inheritance-diagram'}, page_content='manim.camera.camera\nmanim.camera.mapping_camera\nmanim.camera.moving_camera\nmanim.camera.multi_camera\nmanim.camera.three_d_camera\n:parts: 1\n:top-classes: manim.camera.camera.Camera, manim.mobject.mobject.Mobject'),
 Document(metadata={'type': 'summary', 'supplement': 'inheritan

### Testing retrieval after uploading all docs

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
TABLE_NAME = os.getenv("TABLE_NAME")
VECTOR_SIZE = os.getenv("VECTOR_SIZE")

CONNECTION_STRING = (
    f"postgresql+asyncpg://{POSTGRES_USER}:{POSTGRES_HOST}"
    f":{POSTGRES_PORT}/{POSTGRES_DB}"
)

from langchain_postgres import PGEngine

pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)

In [3]:
from backend.workflow.utils.hf_space_wrapper import CustomEmbedding

hf_space = CustomEmbedding()  # embedding service

from langchain_postgres import PGVectorStore

store = await PGVectorStore.create(
    engine=pg_engine,
    table_name=TABLE_NAME,
    # schema_name=SCHEMA_NAME,
    embedding_service=hf_space,
)

In [7]:
def get_detailed_instruct(query: str) -> str:
    task_description="""
Given a query, retrieve relevant code snippets that answer the query
"""
    return f'Instruct: {task_description}\nQuery:{query}'

query = get_detailed_instruct("Create a basic scene where you add hello world text\
 in the screen")

In [ ]:
# k can be added and metadata can be filtered here
docs = await store.asimilarity_search_with_relevance_scores(query)
print(docs)

[(Document(id='a54370e4-af86-41ee-82dd-55dd649cfffc', metadata={'source': '/home/msvelan/Programming/third-party/manim/example_scenes/opengl.py', 'filename': 'opengl.py', 'type': 'code'}, page_content='class InteractiveDevelopment(Scene):\n    def construct(self):\n        circle = Circle()\n        circle.set_fill(BLUE, opacity=0.5)\n        circle.set_stroke(BLUE_E, width=4)\n        square = Square()\n\n        self.play(Create(square))\n        self.wait()\n\n        # This opens an iPython terminal where you can keep writing\n        # lines as if they were part of this construct method.\n        # In particular, \'square\', \'circle\' and \'self\' will all be\n        # part of the local namespace in that terminal.\n        # self.embed()\n\n        # Try copying and pasting some of the lines below into\n        # the interactive shell\n        self.play(ReplacementTransform(square, circle))\n        self.wait()\n        self.play(circle.animate.stretch(4, 0))\n        self.play(

In [12]:
print(docs[0][0].metadata)

{'source': '/home/msvelan/Programming/third-party/manim/example_scenes/opengl.py', 'filename': 'opengl.py', 'type': 'code'}


In [3]:
def pretty_print_docs(docs):
    for i, doc in enumerate(docs, 1):
        print("="*30, f"Block {i}", "="*30)
        print(doc.page_content)
        print("\nMetadata:")
        for k, v in doc.metadata.items():
            print(f"{k}: {v}")

def pretty_print_docs_with_scores(docs):
    for i, (doc, score) in enumerate(docs, 1):
        print("="*30, f"Block {i}", "="*30)
        print(f"Relevance Score: {score}\n")
        print(doc.page_content)
        print("\nMetadata:")
        for k, v in doc.metadata.items():
            print(f"{k}: {v}")

In [14]:
pretty_print_docs_with_scores(docs)

============================== Block 1 ==============================
Relevance Score: 0.6120580434799369

class InteractiveDevelopment(Scene):
    def construct(self):
        circle = Circle()
        circle.set_fill(BLUE, opacity=0.5)
        circle.set_stroke(BLUE_E, width=4)
        square = Square()

        self.play(Create(square))
        self.wait()

        # This opens an iPython terminal where you can keep writing
        # lines as if they were part of this construct method.
        # In particular, 'square', 'circle' and 'self' will all be
        # part of the local namespace in that terminal.
        # self.embed()

        # Try copying and pasting some of the lines below into
        # the interactive shell
        self.play(ReplacementTransform(square, circle))
        self.wait()
        self.play(circle.animate.stretch(4, 0))
        self.play(Rotate(circle, 90 * DEGREES))
        self.play(circle.animate.shift(2 * RIGHT).scale(0.25))

        # text = Text(
     

In [5]:
async def _get_store():
    import os

    from dotenv import load_dotenv
    load_dotenv()

    POSTGRES_USER = os.getenv("POSTGRES_USER")
    POSTGRES_HOST = os.getenv("POSTGRES_HOST")
    POSTGRES_DB = os.getenv("POSTGRES_DB")
    POSTGRES_PORT = os.getenv("POSTGRES_PORT")
    TABLE_NAME = os.getenv("TABLE_NAME")
    VECTOR_SIZE = os.getenv("VECTOR_SIZE")

    CONNECTION_STRING = (
        f"postgresql+asyncpg://{POSTGRES_USER}:{POSTGRES_HOST}"
        f":{POSTGRES_PORT}/{POSTGRES_DB}"
    )

    from langchain_postgres import PGEngine

    pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)
    from backend.workflow.utils.hf_space_wrapper import CustomEmbedding
    hf_space = CustomEmbedding()  # embedding service

    from langchain_postgres import PGVectorStore

    store = await PGVectorStore.create(
        engine=pg_engine,
        table_name=TABLE_NAME,
        # schema_name=SCHEMA_NAME,
        embedding_service=hf_space,
    )
    return store

async def _get_pg_engine():
    import os

    from dotenv import load_dotenv
    load_dotenv()

    POSTGRES_USER = os.getenv("POSTGRES_USER")
    POSTGRES_HOST = os.getenv("POSTGRES_HOST")
    POSTGRES_DB = os.getenv("POSTGRES_DB")
    POSTGRES_PORT = os.getenv("POSTGRES_PORT")

    CONNECTION_STRING = (
        f"postgresql+asyncpg://{POSTGRES_USER}:{POSTGRES_HOST}"
        f":{POSTGRES_PORT}/{POSTGRES_DB}"
    )

    from langchain_postgres import PGEngine

    pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)
    return pg_engine


In [ ]:
async def retrieve_docs(query, k=3, type="code"):
    """Retrieve documents of specific type for a given query"""
    from backend.workflow.utils.hf_space_wrapper import CustomEmbedding
    from langchain_core.documents import Document
    from sqlalchemy import text

    hf_space = CustomEmbedding()

    pg_engine = await _get_pg_engine()

    query_embedding = hf_space.embed_query(query)
    embedding_str = f"[{','.join(map(str, query_embedding))}]"

    sql_query = """
    select langchain_id, content, embedding, langchain_metadata,
    (embedding <=> CAST(:query_embedding AS vector)) as distance
    from public.manim_docs
    where langchain_metadata->>'type'=:type
    order by distance
    limit :k;
    """

    async with pg_engine._pool.connect() as conn:
        result = await conn.execute(text(sql_query), {
            "query_embedding": embedding_str,
            "type": type,
            "k":k
        })
        rows = result.fetchall()

    docs = []
    for row in rows:
        doc = Document(
            page_content=row.content,
            metadata=row.langchain_metadata
        )
        similarity_score = 1 - row.distance
        docs.append((doc, similarity_score))

    return docs

async def get_all_docs(query):
    code_docs = await retrieve_docs(query, k=3, type="code")
    docstrings = await retrieve_docs(query, k=2, type="documentation")
    summary_docs = await retrieve_docs(query, k=2, type="summary")
    return code_docs + docstrings + summary_docs

In [45]:
import nest_asyncio

nest_asyncio.apply()

docs = await get_all_docs(query)

In [50]:
for doc, score in docs:
    print(score)
    print(doc.metadata)

0.6120580434799369
{'source': '/home/msvelan/Programming/third-party/manim/example_scenes/opengl.py', 'filename': 'opengl.py', 'type': 'code'}
0.6025508168400351
{'source': '/home/msvelan/Programming/third-party/manim/docs/source/guides/configuration.rst', 'filename': 'configuration.rst', 'type': 'code', 'ref_classes': '', 'ref_functions': '', 'ref_methods': ''}
0.5676751298129868
{'source': '/home/msvelan/Programming/third-party/manim/example_scenes/opengl.py', 'filename': 'opengl.py', 'type': 'code'}
0.528327051201711
{'source': '/home/msvelan/Programming/third-party/manim/docs/source/tutorials/building_blocks.rst', 'filename': 'building_blocks.rst', 'type': 'documentation', 'title': "Manim's building blocks"}
0.4593751704320487
{'source': '/home/msvelan/Programming/third-party/manim/docs/source/tutorials/building_blocks.rst', 'filename': 'building_blocks.rst', 'type': 'documentation', 'title': "Manim's building blocks"}
0.5390312718190317
{'type': 'summary', 'supplement': 'autosumma

### Testing after integration

In [1]:
import nest_asyncio

from backend.workflow.utils.retrieve import retrieve_docs

nest_asyncio.apply()

docs = await retrieve_docs("Create a square of side length 2 centered at the origin to represent the bounding box of the unit circle.")

In [4]:
pretty_print_docs_with_scores(docs)

============================== Block 1 ==============================
Relevance Score: 0.5635021664799561

References:
  :ref_classes: MathTex Circle Dot Line VGroup
  :ref_methods: Mobject.add_updater Mobject.remove_updater
  :ref_functions: always_redraw
Code:
class SineCurveUnitCircle(Scene):
    # contributed by heejin_park, https://infograph.tistory.com/230
    def construct(self):
        self.show_axis()
        self.show_circle()
        self.move_dot_and_draw_curve()
        self.wait()

    def show_axis(self):
        x_start = np.array([-6,0,0])
        x_end = np.array([6,0,0])

        y_start = np.array([-4,-2,0])
        y_end = np.array([-4,2,0])

        x_axis = Line(x_start, x_end)
        y_axis = Line(y_start, y_end)

        self.add(x_axis, y_axis)
        self.add_x_labels()

        self.origin_point = np.array([-4,0,0])
        self.curve_start = np.array([-3,0,0])

    def add_x_labels(self):
        x_labels = [
            MathTex(r"\pi"), MathTex(r"2 \pi"